# Data Science with Python and Dask
## Chapter 5: Cleaning and Transforming DataFrames

In [1]:
# Before beginning, set your working directory to where the data resides
import os
os.chdir('/Users/Abba/Documents/data-science-python-dask')

### Intro Section

In [2]:
# Listing 5.1
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

#import numpy as np
#import pandas as pd
#pd.set_option("string_storage", "pyarrow")
dask.config.set({"use_nullable_dtypes": True})
dask.config.set({"dataframe.convert-string": True}) # check if this is necessary, or?
dask.config.set({"dataframe.dtype_backend": "pyarrow"})

# start off with dtypes in dask set to pyarrow dtypes. Smaller strings and consistent with pyarrow.to_parquet()

dtypes = {
 'Date First Observed': 'string',
 'Days Parking In Effect    ': 'string',
 'Double Parking Violation': 'string',
 'Feet From Curb': 'float32',
 'From Hours In Effect': 'string',
 'House Number': 'string',
 'Hydrant Violation': 'string',
 'Intersecting Street': 'string',
 'Issue Date': 'string',
 'Issuer Code': 'float32',
 'Issuer Command': 'string',
 'Issuer Precinct': 'float32',
 'Issuer Squad': 'string',
 'Issuing Agency': 'string',
 'Law Section': 'float32',
 'Meter Number': 'string',
 'No Standing or Stopping Violation': 'string',
 'Plate ID': 'string',
 'Plate Type': 'string',
 'Registration State': 'string',
 'Street Code1': 'uint32',
 'Street Code2': 'uint32',
 'Street Code3': 'uint32',
 'Street Name': 'string',
 'Sub Division': 'string',
 'Summons Number': 'uint32',
 'Time First Observed': 'string',
 'To Hours In Effect': 'string',
 'Unregistered Vehicle?': 'string',
 'Vehicle Body Type': 'string',
 'Vehicle Color': 'string',
 'Vehicle Expiration Date': 'string',
 'Vehicle Make': 'string',
 'Vehicle Year': 'float32',
 'Violation Code': 'uint16',
 'Violation County': 'string',
 'Violation Description': 'string',
 'Violation In Front Of Or Opposite': 'string',
 'Violation Legal Code': 'string',
 'Violation Location': 'string',
 'Violation Post Code': 'string',
 'Violation Precinct': 'float32',
 'Violation Time': 'string'
}

nyc_data_raw = dd.read_csv('nyc-parking-tickets/*.csv', dtype=dtypes, usecols=dtypes.keys())
# for a smaller data set for testing:
#nyc_data_raw = dd.read_csv('nyc-parking-tickets/*.csv', dtype=dtypes, usecols=dtypes.keys())

# use this to create a faster parquet raw source directory
nyc_data_raw.to_parquet('nyc_data_raw_parquet', engine='pyarrow', compression='snappy')

In [3]:
# then source parquet instead of csv
nyc_data_raw = dd.read_parquet('nyc_data_raw_parquet')

# TESTING ONLY! write a set of csv files for using smaller test files
with ProgressBar():
    nyc_data_raw.to_csv('nyc-parking-tickets/dask-csv/*.csv')

# delete most of the files and then
# read smaller files
nyc_data_raw = dd.read_csv('nyc-parking-subset-testing/*.csv', dtype=dtypes, usecols=dtypes.keys())

### Section 5.1.1

In [4]:
# Listing 5.2
with ProgressBar():
    display(nyc_data_raw['Plate ID'].head())

[########################################] | 100% Completed | 305.96 ms


0    GBB9093
1    62416MB
2    78755JZ
3    63009MA
4    91648MC
Name: Plate ID, dtype: string

In [5]:
# Listing 5.3
with ProgressBar():
    display(nyc_data_raw[['Plate ID', 'Registration State']].head())

[########################################] | 100% Completed | 408.07 ms


,Plate ID,Registration State
0,GBB9093,NY
1,62416MB,NY
2,78755JZ,NY
3,63009MA,NY
4,91648MC,NY


In [6]:
# Listing 5.4
columns_to_select = ['Plate ID', 'Registration State']

with ProgressBar():
    display(nyc_data_raw[columns_to_select].head())

[########################################] | 100% Completed | 406.22 ms


,Plate ID,Registration State
0,GBB9093,NY
1,62416MB,NY
2,78755JZ,NY
3,63009MA,NY
4,91648MC,NY


### Section 5.1.2

In [7]:
# Listing 5.5
with ProgressBar():
    display(nyc_data_raw.drop('Violation Code', axis=1).head())

[########################################] | 100% Completed | 609.55 ms


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1283294138,GBB9093,NY,PAS,2013-08-04,SUBN,AUDI,P,37250,13610,...,GY,0,2013.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
1,1283294151,62416MB,NY,COM,2013-08-04,VAN,FORD,P,37290,40404,...,WH,0,2012.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
2,1283294163,78755JZ,NY,COM,2013-08-05,P-U,CHEVR,P,37030,31190,...,<NA>,0,0.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
3,1283294175,63009MA,NY,COM,2013-08-05,VAN,FORD,P,37270,11710,...,WH,0,2010.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
4,1283294187,91648MC,NY,COM,2013-08-08,TRLR,GMC,P,37240,12010,...,BR,0,2012.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>


In [8]:
# Listing 5.6
violationColumnNames = list(filter(lambda columnName: 'Violation' in columnName, nyc_data_raw.columns))

with ProgressBar():
    display(nyc_data_raw.drop(violationColumnNames, axis=1).head())

[########################################] | 100% Completed | 509.67 ms


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Law Section,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb
0,1283294138,GBB9093,NY,PAS,2013-08-04,SUBN,AUDI,P,37250,13610,...,408.0,F1,BBBBBBB,ALL,ALL,GY,0,2013.0,-,0.0
1,1283294151,62416MB,NY,COM,2013-08-04,VAN,FORD,P,37290,40404,...,408.0,C,BBBBBBB,ALL,ALL,WH,0,2012.0,-,0.0
2,1283294163,78755JZ,NY,COM,2013-08-05,P-U,CHEVR,P,37030,31190,...,408.0,F7,BBBBBBB,ALL,ALL,<NA>,0,0.0,-,0.0
3,1283294175,63009MA,NY,COM,2013-08-05,VAN,FORD,P,37270,11710,...,408.0,F1,BBBBBBB,ALL,ALL,WH,0,2010.0,-,0.0
4,1283294187,91648MC,NY,COM,2013-08-08,TRLR,GMC,P,37240,12010,...,408.0,E1,BBBBBBB,ALL,ALL,BR,0,2012.0,-,0.0


### Section 5.1.3

In [9]:
# Listing 5.7
nyc_data_renamed = nyc_data_raw.rename(columns={'Plate ID':'License Plate'})
nyc_data_renamed

,Summons Number,License Plate,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
npartitions=138,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,uint32,string,string,string,date32[day][pyarrow],uint16,string,string,string,uint32,uint32,uint32,string,string,float32,float32,float32,string,string,string,string,string,string,string,string,string,string,float32,string,string,string,string,string,string,string,float32,string,float32,string,string,string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### Section 5.1.4

In [10]:
# Listing 5.8
with ProgressBar():
    display(nyc_data_raw.loc[56].head(1))

[########################################] | 100% Completed | 409.38 ms


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
56,1293090530,GES3519,NY,PAS,2013-07-07,40,SDN,HONDA,F,70630,...,BLACK,0,1997.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>


In [11]:
# Listing 5.9
with ProgressBar():
    display(nyc_data_raw.loc[100:200].head(100))

[########################################] | 100% Completed | 407.30 ms


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
100,1294727205,XBD7628,VA,PAS,2013-08-04,17,SUBN,JEEP,P,14510,...,GRY,0,0.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
101,1294727461,R613159,IL,PAS,2013-07-17,17,SDN,VOLKS,P,14510,...,BLUE,0,0.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
102,1294727473,6TCX735,CA,PAS,2013-07-18,17,SDN,MAZDA,P,14510,...,BLACK,0,0.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
103,1294727497,ZWZ43K,NJ,PAS,2013-08-10,17,SUBN,LINCO,P,14510,...,<NA>,0,0.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
104,1295357240,T624858C,NY,PAS,2013-07-22,21,SUBN,TOYOT,X,28790,...,SILVE,0,2012.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1307953700,481CKW,NJ,PAS,2013-07-27,24,SUBN,NISSA,K,0,...,PURPL,0,0.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
196,1307953724,W122KP,NY,PAS,2013-07-28,20,SDN,TOYOT,K,0,...,MAROO,0,1998.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
197,1307953761,GAW6458,NY,PAS,2013-08-17,20,SDN,TOYOT,K,48502,...,BLACK,0,2001.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
198,1307953852,GFP6251,NY,PAS,2013-08-24,27,SDN,NISSA,K,0,...,G/Y,0,2006.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>


In [12]:
# Listing 5.10
with ProgressBar():
    some_rows = nyc_data_raw.loc[100:200].head(100)
some_rows.drop(range(100, 200, 2))

[########################################] | 100% Completed | 409.02 ms


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
101,1294727461,R613159,IL,PAS,2013-07-17,17,SDN,VOLKS,P,14510,...,BLUE,0,0.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
103,1294727497,ZWZ43K,NJ,PAS,2013-08-10,17,SUBN,LINCO,P,14510,...,<NA>,0,0.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
105,1295546486,42909JM,NY,COM,2013-07-17,19,DELV,MERCU,P,58590,...,BROWN,0,1996.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
107,1302446484,452WW4,MA,PAS,2013-06-24,46,SDN,CHEVR,C,75530,...,YELLO,0,2013.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
109,1302453774,GEX5296,NY,PAS,2013-07-26,14,SDN,ME/BE,C,36420,...,BLACK,0,0.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
111,1302453798,867ZFX,CT,PAS,2013-07-15,17,SDN,ACURA,C,0,...,WHITE,0,0.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
113,1302456593,GEZ1408,NY,PAS,2013-07-15,17,SUBN,HYUND,X,77730,...,BLK,0,2013.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
115,1302456738,UCSLIM,NY,PAS,2013-08-07,17,SUBN,FORD,X,77730,...,BLK,0,2003.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
117,1302466203,HPM2636,PA,PAS,2013-08-08,40,SUBN,CADIL,X,24245,...,SILVR,0,0.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
119,1302466227,TWODAMAX,NY,SRF,2013-08-05,17,SDN,HYUND,X,53630,...,GRAY,0,2012.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>


### Section 5.2.1

In [13]:
# Listing 5.11
missing_values = nyc_data_raw.isnull().sum()
with ProgressBar():
    percent_missing = ((missing_values / nyc_data_raw.index.size) * 100).compute()
percent_missing

[########################################] | 100% Completed | 9.85 ss


Summons Number                        0.000000
Plate ID                              0.020867
Registration State                    0.000000
Plate Type                            0.000000
Issue Date                            0.000000
Violation Code                        0.000000
Vehicle Body Type                     0.564922
Vehicle Make                          0.650526
Issuing Agency                        0.000000
Street Code1                          0.000000
Street Code2                          0.000000
Street Code3                          0.000000
Vehicle Expiration Date               0.000002
Violation Location                   15.142846
Violation Precinct                    0.000002
Issuer Precinct                       0.000002
Issuer Code                           0.000002
Issuer Command                       15.018851
Issuer Squad                         15.022566
Violation Time                        0.019207
Time First Observed                  90.040886
Violation Cou

### Section 5.2.2

In [14]:
# Listing 5.12
columns_to_drop = list(percent_missing[percent_missing >= 50].index)
nyc_data_clean_stage1 = nyc_data_raw.drop(columns_to_drop, axis=1)

In [15]:
print(columns_to_drop)

['Time First Observed', 'Intersecting Street', 'Violation Legal Code', 'Unregistered Vehicle?', 'Meter Number', 'No Standing or Stopping Violation', 'Hydrant Violation', 'Double Parking Violation']


### Section 5.2.3

In [16]:
# Listing 5.13
with ProgressBar():
    count_of_vehicle_colors = nyc_data_clean_stage1['Vehicle Color'].value_counts().compute()
most_common_color = count_of_vehicle_colors.sort_values(ascending=False).index[0]

# Fill missing vehicle color with the most common color
nyc_data_clean_stage2 = nyc_data_clean_stage1.fillna({'Vehicle Color': most_common_color})

[########################################] | 100% Completed | 1.07 sms


### Section 5.2.4

In [17]:
# Listing 5.14
# from the bug report, less than 9 rows can work; issue maybe was resolved by 2022.
# Updated to compensate for bug identified in https://github.com/dask/dask/issues/5854

# Old code:
rows_to_drop = list(percent_missing[(percent_missing > 0) & (percent_missing < 5)].index)
nyc_data_clean_stage3 = nyc_data_clean_stage2.dropna(subset=rows_to_drop)

# New code splits the rows to drop into two separate lists and chains the dropna methods to drop all the columns we want
#rows_to_drop1 =['Plate ID', 'Vehicle Body Type', 'Vehicle Make', 'Vehicle Expiration Date', 'Violation Precinct', 'Issuer Precinct', 'Issuer Code', 'Violation Time', 'Street Name']
#rows_to_drop2 =['Date First Observed', 'Law Section', 'Sub Division', 'Vehicle Color', 'Vehicle Year', 'Feet From Curb']
#nyc_data_clean_stage3 = nyc_data_clean_stage2.dropna(subset=rows_to_drop1).dropna(subset=rows_to_drop2)

### Section 5.2.5

In [18]:
# Listing 5.15
remaining_columns_to_clean = list(percent_missing[(percent_missing >= 5) & (percent_missing < 50)].index)
nyc_data_raw.dtypes[remaining_columns_to_clean]

Violation Location                   string[pyarrow]
Issuer Command                       string[pyarrow]
Issuer Squad                         string[pyarrow]
Violation County                     string[pyarrow]
Violation In Front Of Or Opposite    string[pyarrow]
House Number                         string[pyarrow]
Days Parking In Effect               string[pyarrow]
From Hours In Effect                 string[pyarrow]
To Hours In Effect                   string[pyarrow]
Violation Post Code                  string[pyarrow]
Violation Description                string[pyarrow]
dtype: object

In [19]:
# Listing 5.16
unknown_default_dict = dict(map(lambda columnName: (columnName, 'Unknown'), remaining_columns_to_clean))

In [20]:
# Listing 5.17
nyc_data_clean_stage4 = nyc_data_clean_stage3.fillna(unknown_default_dict)

In [21]:
# Listing 5.18
with ProgressBar():
    print(nyc_data_clean_stage4.isnull().sum().compute())
    nyc_data_clean_stage4.persist()

[########################################] | 100% Completed | 24.00 s
Summons Number                       0
Plate ID                             0
Registration State                   0
Plate Type                           0
Issue Date                           0
Violation Code                       0
Vehicle Body Type                    0
Vehicle Make                         0
Issuing Agency                       0
Street Code1                         0
Street Code2                         0
Street Code3                         0
Vehicle Expiration Date              0
Violation Location                   0
Violation Precinct                   0
Issuer Precinct                      0
Issuer Code                          0
Issuer Command                       0
Issuer Squad                         0
Violation Time                       0
Violation County                     0
Violation In Front Of Or Opposite    0
House Number                         0
Street Name                      

nyc_data_clean_stage4.dtypes['Issue Date']

nyc_data_clean_stage4.visualize()

### Section 5.3

In [22]:
# Listing 5.19
with ProgressBar():
    license_plate_types = nyc_data_clean_stage4['Plate Type'].value_counts().compute()
license_plate_types

[########################################] | 100% Completed | 25.38 s


Plate Type
LMB      168
RGC      860
THC      641
CMB    56175
SPC     1133
       ...  
STG      286
WUG       13
NYS      108
STA     2355
SNO        2
Name: count, Length: 90, dtype: int64[pyarrow]

In [23]:
# Listing 5.20
condition = nyc_data_clean_stage4['Plate Type'].isin(['PAS', 'COM'])
plate_type_masked = nyc_data_clean_stage4['Plate Type'].where(condition, 'Other')
nyc_data_recode_stage1 = nyc_data_clean_stage4.drop('Plate Type', axis=1)
nyc_data_recode_stage2 = nyc_data_recode_stage1.assign(PlateType=plate_type_masked)
nyc_data_recode_stage3 = nyc_data_recode_stage2.rename(columns={'PlateType':'Plate Type'})

In [24]:
# Listing 5.21
with ProgressBar():
    display(nyc_data_recode_stage3['Plate Type'].value_counts().compute())

[########################################] | 100% Completed | 11.47 s


Plate Type
Other     3418586
PAS      30452502
COM       7966914
Name: count, dtype: int64[pyarrow]

In [25]:
# Listing 5.22
single_color = list(count_of_vehicle_colors[count_of_vehicle_colors == 1].index)
condition = nyc_data_clean_stage4['Vehicle Color'].isin(single_color)
vehicle_color_masked = nyc_data_clean_stage4['Vehicle Color'].mask(condition, 'Other')
nyc_data_recode_stage4 = nyc_data_recode_stage3.drop('Vehicle Color', axis=1)
nyc_data_recode_stage5 = nyc_data_recode_stage4.assign(VehicleColor=vehicle_color_masked)
nyc_data_recode_stage6 = nyc_data_recode_stage5.rename(columns={'VehicleColor':'Vehicle Color'})

nyc_data_recode_stage6['Issue Date'] = nyc_data_recode_stage6['Issue Date'].astype('string[pyarrow]')

### Section 5.4

# Listing 5.23
from datetime import datetime
issue_date_parsed = nyc_data_recode_stage6['Issue Date'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y"), meta=datetime)
nyc_data_derived_stage1 = nyc_data_recode_stage6.drop('Issue Date', axis=1)
nyc_data_derived_stage2 = nyc_data_derived_stage1.assign(IssueDate=issue_date_parsed)
nyc_data_derived_stage3 = nyc_data_derived_stage2.rename(columns={'IssueDate':'Issue Date'})

In [26]:
nyc_data_derived_stage3 = nyc_data_recode_stage6 # necessary because using parquet raw with autodate

# Listing 5.24
with ProgressBar():
    display(nyc_data_derived_stage3['Issue Date'].head())

nyc_data_derived_stage3['Issue Date'].head().dtype

nyc_data_derived_stage3["Issue Date"] = nyc_data_derived_stage3["Issue Date"].astype("date64[pyarrow]")

nyc_data_derived_stage3['Issue Date'].head().dtype

In [28]:
# Listing 5.25
issue_date_month_year = nyc_data_derived_stage3['Issue Date'].apply(lambda dt: dt.strftime("%Y%m"), meta=int32)
nyc_data_derived_stage4 = nyc_data_derived_stage3.assign(IssueMonthYear=issue_date_month_year)
nyc_data_derived_stage5 = nyc_data_derived_stage4.rename(columns={'IssueMonthYear':'Citation Issued Month Year'})


In [29]:
# Listing 5.26
with ProgressBar():
    display(nyc_data_derived_stage5['Citation Issued Month Year'].head())

[########################################] | 100% Completed | 913.86 ms


0    201308
1    201308
2    201308
3    201308
4    201308
Name: Citation Issued Month Year, dtype: object

### Section 5.5.1

In [30]:
# Listing 5.27
months = ['201310','201410','201510','201610','201710']
condition = nyc_data_derived_stage5['Citation Issued Month Year'].isin(months)
october_citations = nyc_data_derived_stage5[condition]

with ProgressBar():
    display(october_citations.head())

[########################################] | 100% Completed | 5.01 ss


,Summons Number,Plate ID,Registration State,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,Plate Type,Vehicle Color,Citation Issued Month Year
1609,1340313923,GEK8055,NY,2013-10-23,40,SUBN,HONDA,P,79630,40404,...,BBBBBBB,ALL,ALL,2013.0,5.0,Unknown,Unknown,PAS,BROWN,201310
23367,1351679867,XE726658,DE,2013-10-21,20,P-U,DODGE,P,90980,0,...,BBBBBBB,ALL,ALL,2002.0,0.0,Unknown,Unknown,PAS,RED,201310
24172,1351805253,42067JM,NY,2013-10-17,14,DELV,FRUEH,P,25630,13610,...,BBBBBBB,ALL,ALL,1999.0,0.0,Unknown,Unknown,COM,WHITE,201310
32902,1355051060,76254JY,NY,2013-10-10,46,DELV,FRUEH,P,68020,26760,...,BBBBBBB,ALL,ALL,2007.0,0.0,Unknown,Unknown,COM,WHITE,201310
32903,1355051071,44125MC,NY,2013-10-10,46,VAN,FORD,P,68020,26490,...,BBBBBBB,ALL,ALL,2011.0,0.0,Unknown,Unknown,COM,WHITE,201310


In [32]:
# Listing 5.28
bound_date = '2016-04-25' # added padding to month
condition = nyc_data_derived_stage5['Issue Date'] > datetime.strptime(bound_date,"%Y-%m-%d" ).date()
citations_after_bound = nyc_data_derived_stage5[condition]

with ProgressBar():
    display(citations_after_bound.head())

[########################################] | 100% Completed | 5.91 ss


,Summons Number,Plate ID,Registration State,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,Plate Type,Vehicle Color,Citation Issued Month Year
3741,1346495701,GCJ8613,NY,2017-08-04,20,SDN,DODGE,X,10210,19210,...,BBBBBBB,ALL,ALL,2010.0,0.0,Unknown,Unknown,PAS,BLK,201708
3748,1346551819,GHJ2373,NY,2019-07-22,20,SUBN,FORD,X,0,0,...,BBBBBBB,ALL,ALL,2010.0,0.0,Unknown,Unknown,PAS,BLK,201907
6905,1348077426,66811MB,NY,2053-08-02,78,IR,FRUEH,P,25680,46320,...,YYYYYYB,0900P,0500A,2007.0,0.0,Unknown,Unknown,COM,WHT,205308
12967,1353305650,91609MC,NY,2016-08-07,46,VAN,FORD,T,24890,18670,...,BBBBBBB,ALL,ALL,2012.0,0.0,Unknown,Unknown,COM,BR,201608
17144,1354617988,54015JV,NY,2032-07-28,78,VAN,FORD,P,38590,50150,...,BBBBBBB,0900P,0500A,2007.0,0.0,Unknown,Unknown,COM,WHITE,203207


### Section 5.5.1

In [33]:
# Listing 5.29
with ProgressBar():
    condition = (nyc_data_derived_stage5['Issue Date'] > datetime.strptime('2014-01-01', "%Y-%m-%d").date()) & (nyc_data_derived_stage5['Issue Date'] <= datetime.strptime('2017-12-31', "%Y-%m-%d").date())
    nyc_data_filtered = nyc_data_derived_stage5[condition]
    nyc_data_new_index = nyc_data_filtered.set_index('Citation Issued Month Year')

In [34]:
#Did set_index work?
%%time
display(nyc_data_new_index.head(5))

,Summons Number,Plate ID,Registration State,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,Plate Type,Vehicle Color
Citation Issued Month Year,,,,,,,,,,,,,,,,,,,,,
201401,1354990730,FCM7922,NY,2014-01-29,20,SUBN,VOLKS,P,64790,51090,...,D1,BBBBBBB,ALL,ALL,2012.0,0.0,Unknown,Unknown,PAS,WH
201401,1375193314,JJE2003,PA,2014-01-15,40,SDN,ACURA,P,0,40404,...,J3,BBBBBBB,ALL,ALL,0.0,0.0,Unknown,Unknown,PAS,WHITE
201401,1372376525,79648JV,NY,2014-01-30,74,VAN,FRUEH,P,17930,10110,...,C,BBBBBBB,ALL,ALL,2006.0,0.0,Unknown,Unknown,COM,BRN
201401,1367375230,62437JM,NY,2014-01-19,46,DELV,INTER,P,12840,19090,...,F1,BBBBBBB,ALL,ALL,2001.0,0.0,Unknown,Unknown,COM,BROWN
201401,1367375230,62437JM,NY,2014-01-19,46,DELV,INTER,P,12840,19090,...,F1,BBBBBBB,ALL,ALL,2001.0,0.0,Unknown,Unknown,COM,BROWN
201401,1377030556,AGG9214,NY,2014-01-03,40,SDN,LEXUS,P,0,40404,...,C,BBBBBBB,ALL,ALL,2013.0,0.0,Unknown,Unknown,PAS,WHT
201401,1377030556,AGG9214,NY,2014-01-03,40,SDN,LEXUS,P,0,40404,...,C,BBBBBBB,ALL,ALL,2013.0,0.0,Unknown,Unknown,PAS,WHT
201401,1378199625,FLA3634,NY,2014-01-02,71,SUBN,HONDA,P,73720,27510,...,J6,BBBBBBB,ALL,ALL,2001.0,0.0,Unknown,Unknown,PAS,GREY
201401,1379294186,GRH7084,NY,2014-01-20,46,SDN,HONDA,P,0,40404,...,F1,BBBBBBB,ALL,ALL,2014.0,0.0,Unknown,Unknown,PAS,BLK


In [ ]:
nyc_data_new_index.dtypes

nyc_data_new_index2 = nyc_data_filtered.set_index('Citation Issued Month Year')
nyc_data_new_index2.head()

nyc_data_new_index2.index

nyc_data_new_index2.columns

nyc_data_new_index = dd.read_parquet('nyc_data_date_index_not_repart')
display(nyc_data_new_index.columns)

with ProgressBar():
    new_index_columns = list(nyc_data_new_index.columns)

dtype_tuples = [(x, 'string') for x in new_index_columns]
schema1 = dict(dtype_tuples)
schema1 # This provides the columns, but sets everything to 'string', so manual corrections will be needed.

import pyarrow as pa
with ProgressBar():
    first_partition = nyc_data_new_index.get_partition(0).compute()
    table = pa.Table.from_pandas(first_partition, preserve_index=False) #will this work with preserve index?
schema2 = table.schema

schema2


nyc_data_new_index.divisions

In [35]:
# Listing 5.30
years = ['2014', '2015', '2016', '2017']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
divisions = [year + month for year in years for month in months]
name_function = lambda x: f"data-{x}.parquet"

with ProgressBar():
    #nyc_data_new_index.to_parquet('nyc_data_date_index', engine='pyarrow', compression='snappy', schema=schema2)
    nyc_data_new_index.repartition(divisions=divisions).to_parquet('nyc_data_date_index', engine='pyarrow', compression='snappy')#, schema=schema2, name_function=name_function)#,
                            #coerce_timestamps="us",
                            #allow_truncated_timestamps=True)
    
nyc_data_new_index = dd.read_parquet('nyc_data_date_index')

[########################################] | 100% Completed | 270.83 s
[########################################] | 100% Completed | 421.88 s


In [38]:
with ProgressBar():
    display(nyc_data_new_index.head())

[########################################] | 100% Completed | 1.82 ss


,Summons Number,Plate ID,Registration State,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,Plate Type,Vehicle Color
Citation Issued Month Year,,,,,,,,,,,,,,,,,,,,,
201401,1376591133,FMT1257,NY,2014-01-08,21,SUBN,VOLKS,S,42630,17380,...,D1,YYYYYYB,0730A,0800A,2003.0,0.0,Unknown,Unknown,PAS,GREEN
201401,1376591133,FMT1257,NY,2014-01-08,21,SUBN,VOLKS,S,42630,17380,...,D1,YYYYYYB,0730A,0800A,2003.0,0.0,Unknown,Unknown,PAS,GREEN
201401,1379869699,36679PC,NY,2014-01-22,46,BUS,MCI,P,0,40404,...,E1,BBBBBBB,ALL,ALL,2005.0,0.0,Unknown,Unknown,Other,WH
201401,1377064499,GDF5103,NY,2014-01-09,38,SDN,NISSA,P,8140,42490,...,E2,BBBBBBB,ALL,ALL,2013.0,0.0,Unknown,Unknown,PAS,BK
201401,1377064499,GDF5103,NY,2014-01-09,38,SDN,NISSA,P,8140,42490,...,E2,BBBBBBB,ALL,ALL,2013.0,0.0,Unknown,Unknown,PAS,BK


### Section 5.6.1

In [57]:
# Listing 5.31
import pandas as pd
nyc_temps = pd.read_csv('nyc-temp-data.csv')
# added a conversion for monthYear to int
nyc_temps['monthYear'] = pd.to_numeric(nyc_temps['monthYear'], downcast='integer', errors='coerce')
# Filtered out only the relevant months from the temperature data to accelerate the join
nyc_temps_filtered = nyc_temps[nyc_temps.monthYear.isin(divisions)]

In [59]:
nyc_temps_indexed = nyc_temps_filtered.set_index(nyc_temps_filtered.monthYear.astype(str))
nyc_data_with_temps = nyc_data_new_index.join(nyc_temps_indexed, how='inner')

with ProgressBar():
    display(nyc_data_with_temps.head(15))

[########################################] | 100% Completed | 1.95 ss


,Summons Number,Plate ID,Registration State,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,From Hours In Effect,To Hours In Effect,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,Plate Type,Vehicle Color,Temp,monthYear
201401,1376591133,FMT1257,NY,2014-01-08,21,SUBN,VOLKS,S,42630,17380,...,0730A,0800A,2003.0,0.0,Unknown,Unknown,PAS,GREEN,28.6,201401
201401,1376591133,FMT1257,NY,2014-01-08,21,SUBN,VOLKS,S,42630,17380,...,0730A,0800A,2003.0,0.0,Unknown,Unknown,PAS,GREEN,28.6,201401
201401,1379869699,36679PC,NY,2014-01-22,46,BUS,MCI,P,0,40404,...,ALL,ALL,2005.0,0.0,Unknown,Unknown,Other,WH,28.6,201401
201401,1377064499,GDF5103,NY,2014-01-09,38,SDN,NISSA,P,8140,42490,...,ALL,ALL,2013.0,0.0,Unknown,Unknown,PAS,BK,28.6,201401
201401,1377064499,GDF5103,NY,2014-01-09,38,SDN,NISSA,P,8140,42490,...,ALL,ALL,2013.0,0.0,Unknown,Unknown,PAS,BK,28.6,201401
201401,1375953126,EPC7045,NY,2014-01-19,71,SUBN,JEEP,P,29450,36420,...,ALL,ALL,2009.0,0.0,Unknown,Unknown,PAS,BLUE,28.6,201401
201401,1375953126,EPC7045,NY,2014-01-19,71,SUBN,JEEP,P,29450,36420,...,ALL,ALL,2009.0,0.0,Unknown,Unknown,PAS,BLUE,28.6,201401
201401,1374866994,GPH8187,NY,2014-01-06,20,SDN,FORD,P,0,28789,...,ALL,ALL,2003.0,0.0,Unknown,Unknown,PAS,GY,28.6,201401
201401,1376773776,ERL7953,NY,2014-01-25,71,SDN,CHRYS,P,40230,36230,...,ALL,ALL,0.0,0.0,Unknown,Unknown,PAS,GREY,28.6,201401
201401,1374155780,CWY2490,NY,2014-01-25,71,SUBN,TOYOT,P,8940,8440,...,ALL,ALL,1995.0,0.0,Unknown,Unknown,PAS,BLUE,28.6,201401


### Section 5.6.2

In [ ]:
# Listing 5.32
fy16 = dd.read_csv('nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2016.csv', dtype=dtypes, usecols=dtypes.keys())
fy17 = dd.read_csv('nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2017.csv', dtype=dtypes, usecols=dtypes.keys())

fy1617 = fy16.append(fy17)

with ProgressBar():
    print(fy16['Summons Number'].count().compute())

with ProgressBar():
    print(fy17['Summons Number'].count().compute())

with ProgressBar():
    print(fy1617['Summons Number'].count().compute())


### Section 5.7.1

In [ ]:
# Listing 5.33
with ProgressBar():
    if not os.path.exists('nyc-final-csv'):
        os.makedirs('nyc-final-csv') 
    nyc_data_with_temps.repartition(npartitions=1).to_csv('nyc-final-csv/part*.csv')

In [ ]:
# Listing 5.33
with ProgressBar():
    if not os.path.exists('nyc-final-csv-compressed'):
        os.makedirs('nyc-final-csv-compressed')
    nyc_data_with_temps.to_csv(
        filename='nyc-final-csv-compressed/*', 
        compression='gzip', 
        sep='|', 
        na_rep='NULL', 
        header=False, 
        index=False)

### Listing 5.7.2

In [60]:
# Listing 5.35
# Added reset_index as later versions of Dask raise an error stating the index column can't be found
with ProgressBar():
    nyc_data_with_temps.reset_index(drop=True).to_parquet('nyc_final', compression='snappy')

[########################################] | 100% Completed | 48.13 ss
